### TP3 
#### EL ATTARI Yassine

## Exercice 1
   
   Créez un programme Spark pour lire les données "houses"  dans de fichier RealEstate.csv,
   regrouper par emplacement, agréger le prix moyen par SQ Ft et trier par prix moyen par SQ Ft.

    L'ensemble de données contient les champs suivants:
    1. MLS: Numéro de service d'inscription multiple pour la maison (ID unique).
    2. Location: ville / village où se trouve la maison. La plupart des emplacements sont
        Comté de San Luis Obispo et nord du comté de Santa Barbara (Santa MariaOrcutt, Lompoc,
        Guadelupe, Los Alamos), mais il y a aussi des endroits hors de la région.
    3. Price: le prix d'inscription le plus récent de la maison (en dollars).
    4. Bedrooms: nombre de chambres.
    5. Bathrooms: nombre de salles de bain.
    6. Size: taille de la maison en pieds carrés.
    7. Price / SQ.ft: prix de la maison au pied carré.
    8. Statut: type de vente. Les types sont représentés dans le jeu de données:  Short Sale, 
        Foreclosure and Regular.

    Chaque champ est séparé par des virgules.

    Exemple de sortie:

    + ---------------- + ----------------- +
    | Localisation     | avg (Prix SQ Ft) |
    + ---------------- + ----------------- +
    | Oceano           | 95,0 |
    | Bradley          | 206,0 |
    | San Luis Obispo  | 359,0 |
    | Santa Ynez       | 491,4 |
    | Cayucos          | 887,0 |
    | ................ | ................. |
    | ................ | ................. |
    | ................ | ................. |
    '' '
    en utilisant RDD et Dataframe

### EX1 : En utilisant RDD

In [91]:
import pyspark
import os
import csv
try:
    sc
except NameError:
    spark = pyspark.sql.SparkSession.builder.master("local[*]").appName("BD Real Estate").getOrCreate()
    sc = spark.sparkContext
try:
    from StringIO import StringIO
except ImportError:
    from io import StringIO

inputfile="file://"+ os.getcwd() +"/RealEstate.csv"
outputdir="file://"+ os.getcwd() +"/output/"

## en utilisant RDD

def loadRecord(line,header,delimiter):
    """Parse a CSV line"""
    input = StringIO(line)
    reader = csv.DictReader(input, delimiter=delimiter, fieldnames=header)
    return next(reader)
delimiter = ","

input = sc.textFile(inputfile)
header = input.first().split(delimiter)
data = input.filter(lambda x: header[0] not in x).map(lambda x:loadRecord(x,header,delimiter))

locationAndPrice = data.map(lambda ligne: (ligne['Location'],ligne['Price SQ Ft']))

preparingData = locationAndPrice.groupByKey().map(lambda x : (x[0], sum([float(i) for i in list(x[1])]), len(list(x[1])), sum([float(i) for i in list(x[1])])/ len(list(x[1]))))

result = preparingData.map(lambda el:(el[0], el[3])).sortBy(lambda a: a[1])

rs = result.take(10)

print(rs)

[('New Cuyama', 34.05), ('Bakersfield', 69.69), ('King City', 71.51333333333334), ('Greenfield', 91.58), ('Santa Margarita', 95.38), ('Soledad', 102.69333333333333), ('Out Of Area', 116.23333333333333), ('Guadalupe', 120.175), ('Coalinga', 124.34285714285714), ('Santa Maria-Orcutt', 147.58871698113194)]


### EX1 : En utilisant Dataframe

In [113]:
from pyspark.sql.functions import *
input_df = spark.read.options(header='true',inferschema='true',delimiter=delimiter).csv("file://"+os.getcwd()+"/RealEstate.csv")
#poo = input_df.select('Location').avg('Price SQ Ft').show(2)
poo = (input_df
.groupBy("Location")
.avg("Price SQ Ft")
.withColumnRenamed("avg(Price SQ Ft)", "avgPrice")
.orderBy(asc("avgPrice")))
print(poo.show(5))
#print(input_df.show(2))

+---------------+-----------------+
|       Location|         avgPrice|
+---------------+-----------------+
|     New Cuyama|            34.05|
|    Bakersfield|            69.69|
|      King City|71.51333333333334|
|     Greenfield|            91.58|
|Santa Margarita|            95.38|
+---------------+-----------------+
only showing top 5 rows

None


## Exercice 2

Le fichier "nasa_19950701.tsv" contient 10000 lignes de journal provenant de l'un des serveurs apache de la NASA pour le 1er juillet 1995.
     Le fichier "nasa_19950801.tsv" contient 10 000 lignes de journal pour le 1er août 1995
     Créez un programme Spark pour générer un nouveau RDD(Dataframe) contenant les hôtes auxquels on accède les DEUX jours.
     Enregistrez le RDD (Dataframe) résultant dans le fichier "out/nasa_logs_same_hosts.csv".

     Exemple de sortie:
     vagrant.vf.mmc.com
     www-a1.proxy.aol.com
     .....

 Gardez à l'esprit que les fichiers journaux d'origine contiennent les lignes d'en-tête suivantes.
    host    logname    time    method    url    response    bytes
 Assurez-vous que les lignes d'en-tête sont supprimées dans le RDD résultant.

### EX2 : En utilisant RDD

In [14]:
import pyspark
import os
import csv
try:
    sc
except NameError:
    spark = pyspark.sql.SparkSession.builder.master("local[*]").appName("BD Nasa").getOrCreate()
    sc = spark.sparkContext
try:
    from StringIO import StringIO
except ImportError:
    from io import StringIO

inputFirstfile="file://"+ os.getcwd() +"/nasa_19950701.tsv"
inputSecondfile="file://"+ os.getcwd() +"/nasa_19950801.tsv"
outputdir="file://"+ os.getcwd() +"/out/nasa_logs_same_hosts.csv"

## en utilisant RDD

def loadRecord(line,header,delimiter):
    """Parse a CSV line"""
    input = StringIO(line)
    reader = csv.DictReader(input, delimiter=delimiter, fieldnames=header)
    return next(reader)
delimiter = ","

# 1ère fichier
input = sc.textFile(inputFirstfile)
header = input.first().split(delimiter)
data = input.filter(lambda x: header[0] not in x).map(lambda x:loadRecord(x,header,delimiter))

# 2ème fichier
input = sc.textFile(inputSecondfile)
header = input.first().split(delimiter)
secondData = input.filter(lambda x: header[0] not in x).map(lambda x:loadRecord(x,header,delimiter))

# traitement pour récupérer une liste de host pour chaque fichier
firstFile = data.map(lambda ligne: ligne['host\tlogname\ttime\tmethod\turl\tresponse\tbytes'].split('\t')[0])
secondFile = secondData.map(lambda ligne: ligne['host\tlogname\ttime\tmethod\turl\tresponse\tbytes'].split('\t')[0])

result = firstFile.intersection(secondFile) 

# préparation des données à sauvegarder 
rows_of_csv=result.map(lambda content:content.split(',')[0])
# enregistrer des données
rows_of_csv.saveAsTextFile(outputdir)

#### Resultat 

##### contenu du fichier part-00000

- alyssa.prodigy.com
- www-a1.proxy.aol.com
- www-d3.proxy.aol.com
- piweba1y.prodigy.com
- www-b2.proxy.aol.com
- www-b3.proxy.aol.com
- www-d4.proxy.aol.com
- koala.melbpc.org.au
- ***suite de fichier***

##### contenu du fichier part-00001

- piweba3y.prodigy.com
- ottgate2.bnr.ca
- wwwproxy.info.au
- magicall.dacom.co.kr
- palona1.cns.hp.com
- www-b5.proxy.aol.com
- ***suite de fichier***

### EX2 : En utilisant Dataframe

In [43]:
from pyspark.sql.functions import *

# afin d'obtenir un dataframe bien structuré je devais d'utiliser sep au lieu de delimiter pour le fichier tsv
first_df = spark.read.options(header='true',inferschema='true',sep=r'\t').csv("file://"+os.getcwd()+"/nasa_19950701.tsv").select('host').distinct()
second_df = spark.read.options(header='true',inferschema='true',sep=r'\t').csv("file://"+os.getcwd()+"/nasa_19950801.tsv").select('host').distinct()

# Jointure
res = first_df.join(second_df, on='host', how='inner')

# chemin du csv
secondOutputdir="file://"+ os.getcwd() +"/out/nasa_logs_same_hosts_second.csv"

# par défaut il génère de nombreuses partitions c'est mieux de limiter la résultat à  1 seul fichier
res.repartition(1).write.csv(secondOutputdir)
print(distinctDF.show(5))



+--------------------+
|                host|
+--------------------+
|        hella.stm.it|
|srv1.freenet.calg...|
|      ntigate.nt.com|
|www-b5.proxy.aol.com|
|piweba2y.prodigy.com|
+--------------------+
only showing top 5 rows

None


##### La même résultat 37 ligne en totale 